<a href="https://colab.research.google.com/github/Binzbo/Bintang/blob/master/1_ant_colony4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


1.	Definisikan Feromon: Tiap jalan antar titik memiliki 'feromon' yang menunjukkan keberhasilan jalur tersebut dalam rute sebelumnya. Semakin tinggi feromon, semakin menarik jalur tersebut untuk dipilih.
2.	Update Feromon: Setelah setiap perjalanan, feromon di-update berdasarkan keberhasilan mengantarkan paket dengan cepat dan ekonomis.
3.	Evaporasi Feromon: Feromon menguap seiring waktu, sehingga solusi yang tidak optimal akan kurang menarik.
4.	Probabilitas Pemilihan Jalur: Semut memilih jalur berdasarkan probabilitas yang ditentukan oleh jumlah feromon.




In [ ]:
import numpy as np
points = {
    'A': (5, 15),
    'B': (30, 10),
    'C': (10, 5),
    'D': (20, 15),
    'E': (10, 25),
    'F': (20, 30),  # Yellow customer (Priority)
    'G': (5, 45),
    'H': (30, 35),  # Starting point
    'I': (25, 45),  # Red customer (Special)
    'J': (40, 10)
}

# Calculate the Euclidean distances between each pair of points
distances = {}
for key1, coord1 in points.items():
    for key2, coord2 in points.items():
        if key1 != key2:
            # Using Euclidean distance formula
            distance = np.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)
            distances[(key1, key2)] = distance

distances  # Display the calculated distances between points


{('A', 'B'): 25.495097567963924,
 ('A', 'C'): 11.180339887498949,
 ('A', 'D'): 15.0,
 ('A', 'E'): 11.180339887498949,
 ('A', 'F'): 21.213203435596427,
 ('A', 'G'): 30.0,
 ('A', 'H'): 32.01562118716424,
 ('A', 'I'): 36.05551275463989,
 ('A', 'J'): 35.35533905932738,
 ('B', 'A'): 25.495097567963924,
 ('B', 'C'): 20.615528128088304,
 ('B', 'D'): 11.180339887498949,
 ('B', 'E'): 25.0,
 ('B', 'F'): 22.360679774997898,
 ('B', 'G'): 43.01162633521314,
 ('B', 'H'): 25.0,
 ('B', 'I'): 35.35533905932738,
 ('B', 'J'): 10.0,
 ('C', 'A'): 11.180339887498949,
 ('C', 'B'): 20.615528128088304,
 ('C', 'D'): 14.142135623730951,
 ('C', 'E'): 20.0,
 ('C', 'F'): 26.92582403567252,
 ('C', 'G'): 40.311288741492746,
 ('C', 'H'): 36.05551275463989,
 ('C', 'I'): 42.720018726587654,
 ('C', 'J'): 30.4138126514911,
 ('D', 'A'): 15.0,
 ('D', 'B'): 11.180339887498949,
 ('D', 'C'): 14.142135623730951,
 ('D', 'E'): 14.142135623730951,
 ('D', 'F'): 15.0,
 ('D', 'G'): 33.54101966249684,
 ('D', 'H'): 22.360679774997898,


In [ ]:
import random

# Ant Colony Optimization parameters
n_ants = 10  # Number of ants to simulate
n_best = 5  # Number of best ants who leave pheromone
n_iterations = 50  # Number of iterations
decay = 0.1  # Rate at which pheromone decays
alpha = 1  # Pheromone importance
beta = 1  # Distance priority

# Initialize pheromones
pheromones = {edge: 1 for edge in distances.keys()}  # Start with a pheromone level of 1 on all paths

def calculate_probabilities(current_point, visited):
    probs = {}
    for point in points:
        if point not in visited:
            edge = (current_point, point)
            pheromone = pheromones[edge]
            prob = (pheromone ** alpha) * ((1.0 / distances[edge]) ** beta)
            probs[point] = prob
    sum_probs = sum(probs.values())
    return {point: prob / sum_probs for point, prob in probs.items()}

def ant_colony_optimization(start_point):
    shortest_path = None
    shortest_distance = float('inf')

    for _ in range(n_iterations):
        all_paths = []
        for _ in range(n_ants):
            visited = [start_point]
            current_point = start_point
            while len(visited) < len(points):
                probabilities = calculate_probabilities(current_point, visited)
                next_point = random.choices(list(probabilities.keys()), weights=probabilities.values(), k=1)[0]
                visited.append(next_point)
                current_point = next_point
            path_distance = sum(distances[(visited[i], visited[i + 1])] for i in range(len(visited) - 1))
            all_paths.append((visited, path_distance))

        all_paths.sort(key=lambda x: x[1])
        top_paths = all_paths[:n_best]

        # Update pheromones
        for path, distance in top_paths:
            for i in range(len(path) - 1):
                edge = (path[i], path[i + 1])
                pheromones[edge] += 10.0 / distance  # Increase is inversely proportional to distance

        # Decay pheromones
        for edge in pheromones:
            pheromones[edge] *= (1 - decay)

        # Check for the shortest found path
        if top_paths[0][1] < shortest_distance:
            shortest_path, shortest_distance = top_paths[0]

    return shortest_path, shortest_distance

# Run ACO starting from point 'H'
aco_result = ant_colony_optimization('H')
aco_result


(['H', 'I', 'G', 'F', 'E', 'A', 'C', 'D', 'B', 'J'], 121.2570384968221)

In [ ]:
def ant_colony_optimization(start_point, n_ants, n_iterations):
    # Initialize variables to store the shortest path and its distance
    shortest_path = None
    shortest_distance = float('inf')

    for _ in range(n_iterations):
        visited_shortest = None
        distance_shortest = float('inf')

        for _ in range(n_ants):
            visited = [start_point]
            current_point = start_point
            while len(visited) < len(points):
                probabilities = calculate_probabilities(current_point, visited)
                next_point = random.choices(list(probabilities.keys()), weights=probabilities.values(), k=1)[0]
                visited.append(next_point)
                current_point = next_point
            path_distance = sum(distances[(visited[i], visited[i + 1])] for i in range(len(visited) - 1))

            # Check if the current path is shorter than the shortest path found so far in this iteration
            if path_distance < distance_shortest:
                visited_shortest = visited.copy()
                distance_shortest = path_distance

        # Check if the shortest path found in this iteration is shorter than the current shortest path
        if distance_shortest < shortest_distance:
            shortest_path = visited_shortest.copy()
            shortest_distance = distance_shortest

    # Return the shortest path and its distance
    return shortest_path, shortest_distance

# Set the parameters
n_ants = 10  # Number of ants to simulate
n_iterations = 50  # Number of iterations

# Run ACO starting from point 'H'
aco_result = ant_colony_optimization('H', n_ants, n_iterations)
aco_result


(['H', 'I', 'G', 'F', 'E', 'A', 'C', 'D', 'B', 'J'], 121.2570384968221)

138.KM

In [3]:
route = ['H', 'I', 'G', 'F', 'E', 'A', 'C', 'D', 'B', 'J']

# Inisialisasi total jarak
total_distance = 0

# Hitung jarak antara dua node berturut-turut dan tambahkan ke total jarak
for i in range(len(route)-1):
    node1 = route[i]
    node2 = route[i+1]
    distance = distances[(node1, node2)]
    total_distance += distance
    print(f"Jarak dari {node1} ke {node2}: {distance}")

print("Total jarak:", total_distance, "km")


Jarak dari H ke I: 11.180339887498949
Jarak dari I ke G: 20.0
Jarak dari G ke F: 21.213203435596427
Jarak dari F ke E: 11.180339887498949
Jarak dari E ke A: 11.180339887498949
Jarak dari A ke C: 11.180339887498949
Jarak dari C ke D: 14.142135623730951
Jarak dari D ke B: 11.180339887498949
Jarak dari B ke J: 10.0
Total jarak: 121.2570384968221 km
